# AI Základy - Hodina 31: Úvod do strojového učení

## Obsah:
1. **Učení s učitelem (Supervised Learning)**
2. **Učení bez učitele (Unsupervised Learning)**
3. **Učení posilováním (Reinforcement Learning)**
4. **Představení Scikit-learn**
5. **Praktické příklady všech typů učení**
6. **Interaktivní aplikace**
7. **Domácí úkol**

## 1. Úvod do strojového učení

Strojové učení je odvětví umělé inteligence, které umožňuje počítačům učit se z dat bez explicitního programování. Místo toho, abychom psali pravidla, necháme algoritmus objevit vzory v datech.

In [ ]:
# Import všech potřebných knihoven
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_regression, make_blobs, make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, mean_squared_error, silhouette_score
import gradio as gr
import warnings
warnings.filterwarnings('ignore')

# Nastavení vizualizace
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

print("Knihovny úspěšně načteny!")

### 1.1 Vizualizace typů strojového učení

In [ ]:
# Vizualizace tří hlavních typů strojového učení
def visualize_ml_types():
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # 1. Supervised Learning
    np.random.seed(42)
    X_supervised, y_supervised = make_classification(
        n_samples=100, n_features=2, n_redundant=0, n_informative=2,
        n_clusters_per_class=1, random_state=42
    )
    
    axes[0].scatter(X_supervised[:, 0], X_supervised[:, 1], 
                   c=y_supervised, cmap='viridis', s=50, edgecolor='black')
    axes[0].set_title('Učení s učitelem\n(Supervised Learning)', fontsize=16, fontweight='bold')
    axes[0].set_xlabel('Příznak 1')
    axes[0].set_ylabel('Příznak 2')
    
    # Přidání popisků
    for i in range(5):
        axes[0].annotate(f'Třída {y_supervised[i]}', 
                        (X_supervised[i, 0], X_supervised[i, 1]),
                        xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # 2. Unsupervised Learning
    X_unsupervised, _ = make_blobs(
        n_samples=100, n_features=2, centers=3, random_state=42
    )
    
    axes[1].scatter(X_unsupervised[:, 0], X_unsupervised[:, 1], 
                   c='gray', s=50, edgecolor='black', alpha=0.6)
    axes[1].set_title('Učení bez učitele\n(Unsupervised Learning)', fontsize=16, fontweight='bold')
    axes[1].set_xlabel('Příznak 1')
    axes[1].set_ylabel('Příznak 2')
    axes[1].text(0.5, 0.02, 'Data bez značek - hledáme strukturu', 
                ha='center', transform=axes[1].transAxes, fontsize=10, style='italic')
    
    # 3. Reinforcement Learning
    # Simulace jednoduchého prostředí
    grid_size = 5
    grid = np.zeros((grid_size, grid_size))
    
    # Agent pozice
    agent_pos = [2, 0]
    goal_pos = [2, 4]
    obstacles = [[1, 2], [3, 2]]
    
    # Vykreslení mřížky
    for i in range(grid_size):
        for j in range(grid_size):
            color = 'white'
            if [i, j] == agent_pos:
                color = 'blue'
            elif [i, j] == goal_pos:
                color = 'green'
            elif [i, j] in obstacles:
                color = 'red'
            
            rect = plt.Rectangle((j, grid_size-i-1), 1, 1, 
                               facecolor=color, edgecolor='black', linewidth=2)
            axes[2].add_patch(rect)
    
    axes[2].set_xlim(0, grid_size)
    axes[2].set_ylim(0, grid_size)
    axes[2].set_aspect('equal')
    axes[2].set_title('Učení posilováním\n(Reinforcement Learning)', fontsize=16, fontweight='bold')
    axes[2].set_xticks([])
    axes[2].set_yticks([])
    
    # Legenda
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='blue', edgecolor='black', label='Agent'),
        Patch(facecolor='green', edgecolor='black', label='Cíl (+odměna)'),
        Patch(facecolor='red', edgecolor='black', label='Překážka (-trest)')
    ]
    axes[2].legend(handles=legend_elements, loc='upper center', 
                  bbox_to_anchor=(0.5, -0.05), ncol=3)
    
    plt.tight_layout()
    plt.show()

visualize_ml_types()

## 2. Učení s učitelem (Supervised Learning)

Učení s učitelem používá označená data (vstup-výstup páry) k naučení modelu, který dokáže předpovídat výstupy pro nové vstupy.

In [ ]:
# Demonstrace klasifikace a regrese
def supervised_learning_demo():
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # KLASIFIKACE
    print("="*60)
    print("KLASIFIKACE - Předpovídání kategorií")
    print("="*60)
    
    # Vytvoření dat pro klasifikaci
    X_class, y_class = make_classification(
        n_samples=200, n_features=2, n_redundant=0, n_informative=2,
        n_clusters_per_class=1, random_state=42
    )
    
    # Rozdělení na trénovací a testovací
    X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
        X_class, y_class, test_size=0.3, random_state=42
    )
    
    # Tři různé klasifikátory
    classifiers = [
        ('Logistická regrese', LogisticRegression()),
        ('Rozhodovací strom', DecisionTreeClassifier(max_depth=5)),
        ('K-nejbližších sousedů', KNeighborsClassifier(n_neighbors=5))
    ]
    
    for idx, (name, clf) in enumerate(classifiers):
        # Trénování
        clf.fit(X_train_c, y_train_c)
        
        # Predikce
        y_pred = clf.predict(X_test_c)
        accuracy = accuracy_score(y_test_c, y_pred)
        
        # Vytvoření mřížky pro rozhodovací hranici
        h = 0.02
        x_min, x_max = X_class[:, 0].min() - 1, X_class[:, 0].max() + 1
        y_min, y_max = X_class[:, 1].min() - 1, X_class[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                            np.arange(y_min, y_max, h))
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        
        # Vizualizace
        axes[0, idx].contourf(xx, yy, Z, alpha=0.4, cmap='viridis')
        axes[0, idx].scatter(X_train_c[:, 0], X_train_c[:, 1], c=y_train_c, 
                           cmap='viridis', edgecolor='black', s=50, label='Trénovací')
        axes[0, idx].scatter(X_test_c[:, 0], X_test_c[:, 1], c=y_test_c, 
                           cmap='viridis', edgecolor='red', s=100, marker='^', label='Testovací')
        axes[0, idx].set_title(f'{name}\nPřesnost: {accuracy:.2%}', fontsize=14)
        axes[0, idx].legend()
        
        print(f"\n{name}: Přesnost = {accuracy:.2%}")
    
    # REGRESE
    print("\n" + "="*60)
    print("REGRESE - Předpovídání spojitých hodnot")
    print("="*60)
    
    # Vytvoření dat pro regresi
    np.random.seed(42)
    X_reg = np.sort(np.random.uniform(0, 10, 100)).reshape(-1, 1)
    y_reg = 2 * X_reg.ravel() + 1 + np.random.normal(0, 2, X_reg.shape[0])
    
    # Přidání nelinearity
    y_reg[X_reg.ravel() > 5] += 10
    
    # Rozdělení
    X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
        X_reg, y_reg, test_size=0.3, random_state=42
    )
    
    # Tři různé regresory
    regressors = [
        ('Lineární regrese', LinearRegression()),
        ('Rozhodovací strom', DecisionTreeRegressor(max_depth=5)),
        ('Polynomiální regrese', LinearRegression())  # Použijeme polynomiální příznaky
    ]
    
    for idx, (name, reg) in enumerate(regressors):
        # Pro polynomiální regresi vytvoříme polynomiální příznaky
        if name == 'Polynomiální regrese':
            from sklearn.preprocessing import PolynomialFeatures
            poly = PolynomialFeatures(degree=3)
            X_train_poly = poly.fit_transform(X_train_r)
            X_test_poly = poly.transform(X_test_r)
            reg.fit(X_train_poly, y_train_r)
            
            # Predikce
            X_plot = np.linspace(0, 10, 300).reshape(-1, 1)
            X_plot_poly = poly.transform(X_plot)
            y_pred_plot = reg.predict(X_plot_poly)
            y_pred = reg.predict(X_test_poly)
        else:
            reg.fit(X_train_r, y_train_r)
            X_plot = np.linspace(0, 10, 300).reshape(-1, 1)
            y_pred_plot = reg.predict(X_plot)
            y_pred = reg.predict(X_test_r)
        
        mse = mean_squared_error(y_test_r, y_pred)
        
        # Vizualizace
        axes[1, idx].scatter(X_train_r, y_train_r, c='blue', 
                           label='Trénovací', alpha=0.6, edgecolor='black')
        axes[1, idx].scatter(X_test_r, y_test_r, c='red', 
                           label='Testovací', alpha=0.6, edgecolor='black', marker='^', s=100)
        axes[1, idx].plot(X_plot, y_pred_plot, 'g-', linewidth=2, label='Predikce')
        axes[1, idx].set_title(f'{name}\nMSE: {mse:.2f}', fontsize=14)
        axes[1, idx].set_xlabel('X')
        axes[1, idx].set_ylabel('Y')
        axes[1, idx].legend()
        
        print(f"\n{name}: MSE = {mse:.2f}")
    
    plt.tight_layout()
    plt.show()

supervised_learning_demo()

## 3. Učení bez učitele (Unsupervised Learning)

Učení bez učitele hledá strukturu a vzory v datech bez označených výstupů.

In [ ]:
# Demonstrace shlukování a redukce dimenzionality
def unsupervised_learning_demo():
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    print("="*60)
    print("SHLUKOVÁNÍ - Hledání skupin v datech")
    print("="*60)
    
    # Vytvoření různých datasetů
    datasets = [
        ('Oddělené shluky', make_blobs(n_samples=200, centers=3, random_state=42)[0]),
        ('Měsíce', make_moons(n_samples=200, noise=0.1, random_state=42)[0]),
        ('Různé hustoty', np.vstack([
            np.random.normal(0, 0.5, (100, 2)),
            np.random.normal(3, 1.5, (100, 2))
        ]))
    ]
    
    # K-means shlukování
    for idx, (name, X) in enumerate(datasets):
        # Standardizace
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # K-means
        kmeans = KMeans(n_clusters=3, random_state=42)
        clusters = kmeans.fit_predict(X_scaled)
        
        # Silhouette skóre
        silhouette = silhouette_score(X_scaled, clusters)
        
        # Vizualizace
        axes[0, idx].scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', 
                           s=50, edgecolor='black', alpha=0.7)
        axes[0, idx].scatter(kmeans.cluster_centers_[:, 0], 
                           kmeans.cluster_centers_[:, 1],
                           c='red', s=200, marker='*', edgecolor='black',
                           label='Centra shluků')
        axes[0, idx].set_title(f'{name}\nSilhouette: {silhouette:.2f}', fontsize=14)
        axes[0, idx].legend()
        
        print(f"\n{name}: Silhouette skóre = {silhouette:.2f}")
    
    # DBSCAN shlukování pro srovnání
    print("\n" + "="*60)
    print("DBSCAN - Shlukování založené na hustotě")
    print("="*60)
    
    for idx, (name, X) in enumerate(datasets):
        # Standardizace
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # DBSCAN
        dbscan = DBSCAN(eps=0.3, min_samples=5)
        clusters = dbscan.fit_predict(X_scaled)
        
        # Počet shluků (bez šumu)
        n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
        n_noise = list(clusters).count(-1)
        
        # Vizualizace
        unique_clusters = set(clusters)
        colors = plt.cm.viridis(np.linspace(0, 1, len(unique_clusters)))
        
        for cluster_id, color in zip(unique_clusters, colors):
            if cluster_id == -1:
                # Šum
                mask = clusters == cluster_id
                axes[1, idx].scatter(X[mask, 0], X[mask, 1], 
                                   c='gray', s=30, alpha=0.3, label='Šum')
            else:
                mask = clusters == cluster_id
                axes[1, idx].scatter(X[mask, 0], X[mask, 1], 
                                   c=[color], s=50, edgecolor='black', 
                                   alpha=0.7, label=f'Shluk {cluster_id}')
        
        axes[1, idx].set_title(f'{name}\nPočet shluků: {n_clusters}, Šum: {n_noise}', 
                              fontsize=14)
        if idx == 0:  # Legenda jen u prvního
            axes[1, idx].legend()
        
        print(f"\n{name}: Nalezeno {n_clusters} shluků, {n_noise} bodů šumu")
    
    plt.tight_layout()
    plt.show()

unsupervised_learning_demo()

### 3.1 Redukce dimenzionality s PCA

In [ ]:
# Demonstrace PCA
def pca_demo():
    # Vytvoření 3D dat
    np.random.seed(42)
    n_samples = 500
    
    # Vytvoření korelovaných dat
    mean = [0, 0, 0]
    cov = [[1, 0.8, 0.6],
           [0.8, 1, 0.7],
           [0.6, 0.7, 1]]
    
    X_3d = np.random.multivariate_normal(mean, cov, n_samples)
    
    # PCA
    pca = PCA()
    X_pca = pca.fit_transform(X_3d)
    
    # Vizualizace
    fig = plt.figure(figsize=(18, 6))
    
    # 1. Původní 3D data
    ax1 = fig.add_subplot(131, projection='3d')
    ax1.scatter(X_3d[:, 0], X_3d[:, 1], X_3d[:, 2], 
               c=np.arange(n_samples), cmap='viridis', alpha=0.6)
    ax1.set_title('Původní 3D data', fontsize=14)
    ax1.set_xlabel('X1')
    ax1.set_ylabel('X2')
    ax1.set_zlabel('X3')
    
    # 2. Vysvětlená variance
    ax2 = fig.add_subplot(132)
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
    
    ax2.bar(range(1, 4), explained_variance_ratio, alpha=0.7, 
           label='Jednotlivé komponenty', color='blue')
    ax2.plot(range(1, 4), cumulative_variance_ratio, 'ro-', 
            linewidth=2, markersize=8, label='Kumulativní')
    
    ax2.set_xlabel('Komponenta')
    ax2.set_ylabel('Vysvětlená variance')
    ax2.set_title('Vysvětlená variance PCA komponent', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 1.1)
    
    # Přidání hodnot
    for i, (var, cum_var) in enumerate(zip(explained_variance_ratio, cumulative_variance_ratio)):
        ax2.text(i+1, var + 0.02, f'{var:.2%}', ha='center', fontsize=10)
        ax2.text(i+1, cum_var + 0.02, f'{cum_var:.2%}', ha='center', fontsize=10)
    
    # 3. Data po PCA (2D)
    ax3 = fig.add_subplot(133)
    ax3.scatter(X_pca[:, 0], X_pca[:, 1], 
               c=np.arange(n_samples), cmap='viridis', alpha=0.6)
    ax3.set_title('Data po redukci na 2D', fontsize=14)
    ax3.set_xlabel('První hlavní komponenta')
    ax3.set_ylabel('Druhá hlavní komponenta')
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("="*60)
    print("ANALÝZA PCA")
    print("="*60)
    print(f"\nVysvětlená variance jednotlivými komponentami:")
    for i, var in enumerate(explained_variance_ratio):
        print(f"Komponenta {i+1}: {var:.2%}")
    print(f"\nPrvní 2 komponenty vysvětlují {cumulative_variance_ratio[1]:.2%} variance")

pca_demo()

## 4. Učení posilováním (Reinforcement Learning)

Agent se učí interakcí s prostředím na základě odměn a trestů.

In [ ]:
# Jednoduchá simulace učení posilováním
class SimpleGridWorld:
    def __init__(self, size=5):
        self.size = size
        self.reset()
        
    def reset(self):
        self.agent_pos = [0, 0]
        self.goal_pos = [4, 4]
        self.obstacles = [[1, 1], [2, 2], [3, 1]]
        self.done = False
        self.total_reward = 0
        return self.agent_pos
    
    def step(self, action):
        # Akce: 0=nahoru, 1=dolů, 2=vlevo, 3=vpravo
        moves = [[-1, 0], [1, 0], [0, -1], [0, 1]]
        
        new_pos = [self.agent_pos[0] + moves[action][0],
                   self.agent_pos[1] + moves[action][1]]
        
        # Kontrola hranic
        if (0 <= new_pos[0] < self.size and 
            0 <= new_pos[1] < self.size and 
            new_pos not in self.obstacles):
            self.agent_pos = new_pos
            reward = -1  # Malý trest za každý krok
        else:
            reward = -5  # Větší trest za náraz
        
        # Kontrola cíle
        if self.agent_pos == self.goal_pos:
            reward = 100  # Velká odměna za dosažení cíle
            self.done = True
        
        self.total_reward += reward
        return self.agent_pos, reward, self.done
    
    def render(self, ax, q_values=None):
        ax.clear()
        
        # Vykreslení mřížky
        for i in range(self.size):
            for j in range(self.size):
                color = 'white'
                if [i, j] == self.agent_pos:
                    color = 'blue'
                elif [i, j] == self.goal_pos:
                    color = 'green'
                elif [i, j] in self.obstacles:
                    color = 'red'
                
                rect = plt.Rectangle((j, self.size-i-1), 1, 1,
                                   facecolor=color, edgecolor='black', linewidth=2)
                ax.add_patch(rect)
                
                # Zobrazení Q-hodnot
                if q_values is not None and [i, j] not in self.obstacles:
                    q_val = q_values.get((i, j), 0)
                    ax.text(j+0.5, self.size-i-0.5, f'{q_val:.1f}',
                           ha='center', va='center', fontsize=8)
        
        ax.set_xlim(0, self.size)
        ax.set_ylim(0, self.size)
        ax.set_aspect('equal')
        ax.set_xticks([])
        ax.set_yticks([])

# Demonstrace Q-learning
def q_learning_demo():
    env = SimpleGridWorld()
    
    # Q-learning parametry
    q_table = {}
    learning_rate = 0.1
    discount_factor = 0.9
    epsilon = 0.1
    n_episodes = 100
    
    # Historie pro vizualizaci
    rewards_history = []
    
    print("="*60)
    print("Q-LEARNING SIMULACE")
    print("="*60)
    print("\nTrénování agenta...")
    
    for episode in range(n_episodes):
        state = tuple(env.reset())
        episode_reward = 0
        
        while not env.done:
            # Epsilon-greedy strategie
            if np.random.random() < epsilon:
                action = np.random.randint(4)
            else:
                # Výběr nejlepší akce podle Q-tabulky
                q_values = [q_table.get((state, a), 0) for a in range(4)]
                action = np.argmax(q_values)
            
            # Provedení akce
            next_state, reward, done = env.step(action)
            next_state = tuple(next_state)
            
            # Q-learning update
            old_q = q_table.get((state, action), 0)
            next_max_q = max([q_table.get((next_state, a), 0) for a in range(4)])
            new_q = old_q + learning_rate * (reward + discount_factor * next_max_q - old_q)
            q_table[(state, action)] = new_q
            
            state = next_state
            episode_reward += reward
        
        rewards_history.append(episode_reward)
        
        if (episode + 1) % 20 == 0:
            print(f"Epizoda {episode + 1}: Celková odměna = {episode_reward}")
    
    # Vizualizace výsledků
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # 1. Historie odměn
    axes[0].plot(rewards_history, linewidth=2)
    axes[0].set_xlabel('Epizoda')
    axes[0].set_ylabel('Celková odměna')
    axes[0].set_title('Učení v čase', fontsize=14)
    axes[0].grid(True, alpha=0.3)
    
    # Klouzavý průměr
    window = 10
    moving_avg = np.convolve(rewards_history, np.ones(window)/window, mode='valid')
    axes[0].plot(range(window-1, len(rewards_history)), moving_avg, 
                'r-', linewidth=2, label=f'{window}-epizodový průměr')
    axes[0].legend()
    
    # 2. Naučená politika
    env_vis = SimpleGridWorld()
    state_q_values = {}
    
    for i in range(env_vis.size):
        for j in range(env_vis.size):
            if [i, j] not in env_vis.obstacles:
                q_vals = [q_table.get(((i, j), a), 0) for a in range(4)]
                state_q_values[(i, j)] = max(q_vals)
    
    env_vis.render(axes[1], state_q_values)
    axes[1].set_title('Naučené Q-hodnoty', fontsize=14)
    
    # 3. Optimální cesta
    env_demo = SimpleGridWorld()
    path = [env_demo.agent_pos.copy()]
    
    for _ in range(20):  # Max 20 kroků
        state = tuple(env_demo.agent_pos)
        q_values = [q_table.get((state, a), 0) for a in range(4)]
        action = np.argmax(q_values)
        env_demo.step(action)
        path.append(env_demo.agent_pos.copy())
        if env_demo.done:
            break
    
    env_demo = SimpleGridWorld()
    env_demo.render(axes[2])
    
    # Vykreslení cesty
    path_array = np.array(path)
    axes[2].plot(path_array[:, 1] + 0.5, 
                env_demo.size - path_array[:, 0] - 0.5,
                'y-', linewidth=3, marker='o', markersize=10,
                markerfacecolor='yellow', markeredgecolor='black')
    axes[2].set_title('Naučená optimální cesta', fontsize=14)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nFinální průměrná odměna (posledních 10 epizod): {np.mean(rewards_history[-10:]):.2f}")

q_learning_demo()

## 5. Představení Scikit-learn

Scikit-learn poskytuje jednotné API pro různé algoritmy strojového učení.

In [ ]:
# Ukázka jednotného API Scikit-learn
def sklearn_api_demo():
    print("="*70)
    print("SCIKIT-LEARN API - Jednotné rozhraní pro všechny algoritmy")
    print("="*70)
    
    # Vytvoření dat
    X, y = make_classification(n_samples=200, n_features=2, 
                              n_redundant=0, n_informative=2,
                              n_clusters_per_class=1, random_state=42)
    
    # Rozdělení dat
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
    
    # Různé modely - všechny používají stejné API
    models = {
        'Logistická regrese': LogisticRegression(),
        'SVM': SVC(),
        'Rozhodovací strom': DecisionTreeClassifier(),
        'KNN': KNeighborsClassifier()
    }
    
    print("\n1. ZÁKLADNÍ WORKFLOW:")
    print("   model = ModelClass()         # Vytvoření modelu")
    print("   model.fit(X_train, y_train)  # Trénování")
    print("   y_pred = model.predict(X_test) # Predikce")
    print("   score = model.score(X_test, y_test) # Vyhodnocení")
    
    print("\n2. VÝSLEDKY RŮZNÝCH MODELŮ:")
    print("-" * 50)
    
    results = []
    
    for name, model in models.items():
        # Jednotné API pro všechny modely
        model.fit(X_train, y_train)              # Trénování
        score = model.score(X_test, y_test)      # Vyhodnocení
        y_pred = model.predict(X_test)           # Predikce
        
        results.append({
            'Model': name,
            'Přesnost': f"{score:.3f}",
            'Trénovací přesnost': f"{model.score(X_train, y_train):.3f}"
        })
        
        print(f"\n{name}:")
        print(f"  - Přesnost na testovacích datech: {score:.3f}")
        print(f"  - Přesnost na trénovacích datech: {model.score(X_train, y_train):.3f}")
    
    # Vytvoření přehledové tabulky
    results_df = pd.DataFrame(results)
    
    print("\n3. SOUHRN VÝSLEDKŮ:")
    print("-" * 50)
    print(results_df.to_string(index=False))
    
    # Vizualizace
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    models_names = results_df['Model']
    test_scores = [float(score) for score in results_df['Přesnost']]
    train_scores = [float(score) for score in results_df['Trénovací přesnost']]
    
    x = np.arange(len(models_names))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, train_scores, width, 
                   label='Trénovací přesnost', color='blue', alpha=0.7)
    bars2 = ax.bar(x + width/2, test_scores, width, 
                   label='Testovací přesnost', color='green', alpha=0.7)
    
    ax.set_xlabel('Model')
    ax.set_ylabel('Přesnost')
    ax.set_title('Porovnání modelů - Jednotné API Scikit-learn', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(models_names, rotation=45, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Přidání hodnot
    for bars, scores in [(bars1, train_scores), (bars2, test_scores)]:
        for bar, score in zip(bars, scores):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                   f'{score:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    print("\n4. DALŠÍ UŽITEČNÉ METODY:")
    print("-" * 50)
    print("   model.predict_proba(X)  # Pravděpodobnosti tříd (pro klasifikaci)")
    print("   model.get_params()      # Získání parametrů modelu")
    print("   model.set_params(**params) # Nastavení parametrů")

sklearn_api_demo()

## 6. Interaktivní aplikace

In [ ]:
# Interaktivní demonstrace všech typů učení
def create_ml_playground():
    def ml_demo(learning_type, algorithm, n_samples):
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        if learning_type == "Supervised - Classification":
            # Vytvoření dat
            X, y = make_classification(n_samples=n_samples, n_features=2,
                                     n_redundant=0, n_informative=2,
                                     n_clusters_per_class=1, random_state=42)
            
            # Výběr modelu
            if algorithm == "Logistic Regression":
                model = LogisticRegression()
            elif algorithm == "Decision Tree":
                model = DecisionTreeClassifier(max_depth=5)
            else:  # KNN
                model = KNeighborsClassifier(n_neighbors=5)
            
            # Trénování
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.3, random_state=42
            )
            model.fit(X_train, y_train)
            accuracy = model.score(X_test, y_test)
            
            # Vizualizace dat
            axes[0].scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', 
                          edgecolor='black', s=50)
            axes[0].set_title('Data', fontsize=12)
            
            # Rozhodovací hranice
            h = 0.02
            x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
            y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
            xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                                np.arange(y_min, y_max, h))
            Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)
            
            axes[1].contourf(xx, yy, Z, alpha=0.4, cmap='viridis')
            axes[1].scatter(X_test[:, 0], X_test[:, 1], c=y_test, 
                          cmap='viridis', edgecolor='black', s=50)
            axes[1].set_title(f'Model: {algorithm}\nPřesnost: {accuracy:.2%}', 
                            fontsize=12)
            
        elif learning_type == "Supervised - Regression":
            # Vytvoření dat
            X = np.sort(np.random.uniform(0, 10, n_samples)).reshape(-1, 1)
            y = 2 * X.ravel() + 1 + np.random.normal(0, 2, n_samples)
            
            # Výběr modelu
            if algorithm == "Linear Regression":
                model = LinearRegression()
            else:  # Decision Tree
                model = DecisionTreeRegressor(max_depth=5)
            
            # Trénování
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.3, random_state=42
            )
            model.fit(X_train, y_train)
            mse = mean_squared_error(y_test, model.predict(X_test))
            
            # Vizualizace
            axes[0].scatter(X, y, alpha=0.6, edgecolor='black')
            axes[0].set_title('Data', fontsize=12)
            
            X_plot = np.linspace(0, 10, 300).reshape(-1, 1)
            y_pred = model.predict(X_plot)
            
            axes[1].scatter(X_test, y_test, alpha=0.6, edgecolor='black', 
                          label='Testovací data')
            axes[1].plot(X_plot, y_pred, 'r-', linewidth=2, label='Model')
            axes[1].set_title(f'Model: {algorithm}\nMSE: {mse:.2f}', fontsize=12)
            axes[1].legend()
            
        else:  # Unsupervised
            # Vytvoření dat
            X, _ = make_blobs(n_samples=n_samples, centers=3, random_state=42)
            
            # K-means
            kmeans = KMeans(n_clusters=3, random_state=42)
            clusters = kmeans.fit_predict(X)
            
            # Vizualizace
            axes[0].scatter(X[:, 0], X[:, 1], alpha=0.6, edgecolor='black', 
                          c='gray')
            axes[0].set_title('Původní data (bez značek)', fontsize=12)
            
            axes[1].scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis',
                          edgecolor='black', alpha=0.6)
            axes[1].scatter(kmeans.cluster_centers_[:, 0],
                          kmeans.cluster_centers_[:, 1],
                          c='red', s=200, marker='*', edgecolor='black')
            axes[1].set_title('K-means shlukování', fontsize=12)
        
        for ax in axes:
            ax.set_xlabel('X1')
            ax.set_ylabel('X2')
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        return fig
    
    # Gradio interface
    with gr.Blocks(title="ML Playground") as demo:
        gr.Markdown("# 🤖 Strojové učení - Interaktivní ukázka")
        gr.Markdown("Experimentujte s různými typy a algoritmy strojového učení!")
        
        with gr.Row():
            with gr.Column():
                learning_type = gr.Radio(
                    choices=["Supervised - Classification", 
                            "Supervised - Regression",
                            "Unsupervised - Clustering"],
                    value="Supervised - Classification",
                    label="Typ učení"
                )
                
                algorithm = gr.Dropdown(
                    choices=["Logistic Regression", "Decision Tree", "KNN"],
                    value="Logistic Regression",
                    label="Algoritmus"
                )
                
                n_samples = gr.Slider(
                    minimum=50,
                    maximum=500,
                    value=200,
                    step=50,
                    label="Počet vzorků"
                )
                
                generate_btn = gr.Button("🚀 Spustit", variant="primary")
            
            with gr.Column():
                output_plot = gr.Plot(label="Vizualizace")
        
        # Aktualizace algoritmů podle typu učení
        def update_algorithms(learning_type):
            if learning_type == "Supervised - Classification":
                return gr.Dropdown(choices=["Logistic Regression", 
                                          "Decision Tree", "KNN"])
            elif learning_type == "Supervised - Regression":
                return gr.Dropdown(choices=["Linear Regression", 
                                          "Decision Tree"])
            else:
                return gr.Dropdown(choices=["K-means"], value="K-means")
        
        learning_type.change(update_algorithms, inputs=[learning_type], 
                           outputs=[algorithm])
        
        generate_btn.click(ml_demo, 
                          inputs=[learning_type, algorithm, n_samples],
                          outputs=[output_plot])
        
        gr.Markdown("""
        ### 📚 Vysvětlení:
        
        **Supervised Learning (Učení s učitelem)**
        - Klasifikace: Předpovídání kategorií (např. spam/ne-spam)
        - Regrese: Předpovídání číselných hodnot (např. cena domu)
        
        **Unsupervised Learning (Učení bez učitele)**
        - Shlukování: Hledání skupin v datech bez předem daných značek
        
        **Reinforcement Learning (Učení posilováním)**
        - Agent se učí pomocí odměn a trestů (není v této ukázce)
        """)
    
    return demo

# Spuštění aplikace
demo = create_ml_playground()
demo.launch(share=True)

## 7. Shrnutí a klíčové koncepty

### Tři hlavní typy strojového učení:

1. **Učení s učitelem (Supervised Learning)**
   - Používá označená data (vstup-výstup páry)
   - Klasifikace: předpovídání kategorií
   - Regrese: předpovídání číselných hodnot

2. **Učení bez učitele (Unsupervised Learning)**
   - Pracuje s neoznačenými daty
   - Shlukování: hledání skupin
   - Redukce dimenzionality: PCA

3. **Učení posilováním (Reinforcement Learning)**
   - Agent se učí interakcí s prostředím
   - Odměny a tresty
   - Q-learning, policy gradient

### Scikit-learn API:
```python
# Jednotný přístup pro všechny algoritmy
model = ModelClass()           # Vytvoření
model.fit(X_train, y_train)    # Trénování
predictions = model.predict(X_test)  # Predikce
score = model.score(X_test, y_test)  # Vyhodnocení
```

## 8. Domácí úkol

### Úkol 1: Porovnání algoritmů
Vytvořte vlastní dataset a porovnejte alespoň 5 různých algoritmů pro:
- Klasifikaci
- Regresi
- Vytvořte vizualizaci výsledků

### Úkol 2: Unsupervised learning
- Najděte optimální počet shluků pomocí elbow metody
- Porovnejte K-means s DBSCAN na různých datasetech
- Implementujte hierarchické shlukování

### Úkol 3: Mini RL projekt
- Rozšiřte GridWorld o více překážek
- Implementujte SARSA algoritmus
- Porovnejte Q-learning a SARSA

### Bonusový úkol: Pipeline
Vytvořte kompletní ML pipeline včetně:
- Předzpracování dat
- Výběru příznaků
- Cross-validace
- Hyperparameter tuningu

---

💡 **Tip**: Scikit-learn dokumentace je vynikající zdroj příkladů a vysvětlení!